In [63]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime
import numpy as np

In [8]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False 
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')     

In [3]:
data = pd.read_csv("../mydata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)

In [9]:
e=['ajo','albaricoque']

In [70]:
driver = "../driver/chromedriver.exe"
driver = webdriver.Chrome(driver, options=opciones)
supermarket = "carrefour"
prices = []
prod = []
sup = []
links = []
names = []
newprice = []
frame = {}
for f in e:
    for i in range(2,4):
        url = f"https://www.carrefour.es/?q={f}&page={i}"
        driver.get(url)
        time.sleep(0.3)
        try: #cookies
            driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
            time.sleep(0.7)
        except:
            pass
        try:
            for o in range(1,50): #price, scroll, product, supermarket, link and name
                    prices.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > span').text.split("€")[0])
                    driver.execute_script("window.scrollTo(0, window.scrollY + 300)")
                    prod.append(f)
                    sup.append('carrefour')
                    links.append(url)
                    names.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > a.ebx-result-link.ebx-result__title-link.track-click > h1').text)
        except:
            pass

frame['price'] = prices
frame['product'] = prod
frame['supermarket'] = sup
frame['link'] = links
frame['name'] = names

scrap = pd.DataFrame(frame)
for i, row in scrap.iterrows():
    if row['price'] == '':
        newprice.append(np.nan)
    else:
        newprice.append(row['price'])
scrap['price'] = newprice
scrap.dropna(how='any',inplace=True)
scrap['price'] = [float((i.replace(",",".").replace("€","").strip())) for i in scrap['price']]


if not os.path.exists(f"../mydata/scraps_{supermarket}"):
    os.makedirs(f"../mydata/scraps_{supermarket}")
    now = str(datetime.now())[:19].replace(":","_")
    scrap.to_csv(f'../mydata/scraps_{supermarket}/{now}.csv')

/tmp/ipykernel_429/277639645.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver, options=opciones)
/tmp/ipykernel_429/277639645.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/tmp/ipykernel_429/277639645.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > span').text.split("€")[0])
/tmp/ipykernel_429/277639645.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  names.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > a.ebx-result-link.ebx-result__title-link.track-click > h1').text)


,price,product,supermarket,link,name
0,"5,80",ajo,carrefour,https://www.carrefour.es/?q=ajo&page=2,Ajo Círculo de Calidad 250 g
1,"4,18",ajo,carrefour,https://www.carrefour.es/?q=ajo&page=2,Ajo sprint 500 g
4,"11,96",albaricoque,carrefour,https://www.carrefour.es/?q=albaricoque&page=2,Orejón albaricoque Carrefour tarrina 250 g
5,"2,41",albaricoque,carrefour,https://www.carrefour.es/?q=albaricoque&page=2,Mermelada de albaricoque categoría extra Carre...
6,"11,96",albaricoque,carrefour,https://www.carrefour.es/?q=albaricoque&page=3,Orejón albaricoque Carrefour tarrina 250 g
7,"2,41",albaricoque,carrefour,https://www.carrefour.es/?q=albaricoque&page=3,Mermelada de albaricoque categoría extra Carre...
